In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/indon.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/texts.txt

In [2]:
import malaya

electra = malaya.transformer.load(model = 'electra')


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.






INFO:tensorflow:Restoring parameters from /home/husein/Malaya/electra-model/base/electra-base/model.ckpt


In [3]:
vocab_wiki, embedded_wiki = malaya.wordvector.load_wiki()
word_vector_wiki = malaya.wordvector.load(embedded_wiki, vocab_wiki)

In [4]:
with open('texts.txt') as fopen:
    texts = fopen.read().split('\n')

with open('indon.txt') as fopen:
    parsing = fopen.read().split('\n')
    
len(texts), len(parsing)

(1030, 1030)

In [5]:
def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

In [15]:
import re

def remove(string):
    string = string.encode('ascii', errors='ignore').strip().decode()
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string
    
def augment(string, n = 7):
    results = malaya.augmentation.synonym(string)
    try:
        results.extend(malaya.augmentation.transformer(string, electra))
    except:
        pass
    results.extend(malaya.augmentation.wordvector(
        string, word_vector_wiki, soft = True
    ))
    results = list(set(results))
    results = [remove(s) for s in results if s.lower() != string.lower()]
    words = string.split()
    results = [s.split() for s in results if len([w for w in s.split() if len(w) > 1]) == len(words)]
    for i in range(len(results)):
        for no, w in enumerate(words):
            results[i][no] = case_of(w)(results[i][no])
        results[i] = ' '.join(results[i])
    return results[:n]

def replace(string, actual_string, aug_string):
    actual_string = actual_string.split()
    aug_string = aug_string.split()
    for no, word in enumerate(actual_string):
        string = string.replace(word, aug_string[no])
    return string

In [16]:
from tqdm import tqdm

results, out = [], []
for i in tqdm(range(len(texts))):
    try:
        rows = augment(texts[i])
        for row in rows:
            results.append(replace(parsing[i], texts[i], row))
            out.append(row)
    except:
        pass

100%|██████████| 1030/1030 [42:50<00:00,  2.50s/it] 


In [17]:
len(results)

4472

In [18]:
with open('augmented.txt', 'w') as fopen:
    fopen.write('\n'.join(results))

In [19]:
with open('augmented-texts.txt', 'w') as fopen:
    fopen.write('\n'.join(out))